# Compare runs

In [17]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt

from osgeo import ogr
import pyvista as pv
import os
import sys
from scipy.spatial import KDTree

from interessant import * # Bei Änderungen Kernel neu starten
sys.path.append("/home/riannek/code/gleis/gleisachse")
from algs.centralpoints import *

ogr.UseExceptions()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
run24 = "/media/riannek/minimax/gleis/run24-2024-08-13"
run14 = "/media/riannek/minimax/gleis/run14-2024-08-14"


# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A: 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37; Auch viele Gleise: 43; Kreuzung: 44, 45; 47 Drei

key = list(interessant.keys())[47] 

filename = interessant[key]
filename = filename.split(".")[0] + ".copc.laz"
print(key, filename)

file_run24 = os.path.join(run24, filename)
file_run14 = os.path.join(run14, filename)

if not os.path.exists(file_run14):
    raise FileNotFoundError(filename)

Drei 4473825_5336300.copc.laz


In [27]:
gauge = 1.435
railhead_width = 0.067

In [25]:
def get_seedpoints(points):
    low_intensity = points[points["Intensity"] < 14500]
    downsampling_pipeline = pdal.Filter("filters.sample", radius=s.seedpoint_distance).pipeline(low_intensity)
    downsampling_pipeline.execute()
    seed_points = downsampling_pipeline.arrays[0]
    return np.vstack((seed_points["X"], seed_points["Y"], seed_points["Z"])).T

In [31]:
def get_points(xyz, xyz_seed): 
    points = []
    tree = KDTree(xyz)
    # indices: ndarray (dtype object) with a list of indices for each seed point
    indices = tree.query_ball_point(xyz_seed, r=s.neighborhood_radius, workers=-1)

    seed_point_count = len(xyz_seed)

    for cluster, clustercenter, cluster_other, center_other, eigenvects in pair_generator(xyz, tree, indices, seed_point_count, gauge):
        transformmatrix = get_transformmatrix(cluster, cluster_other, clustercenter, center_other, eigenvects)
        representive_a, side_a  = representive_point(cluster, transformmatrix, railhead_width) 
        representive_b, side_b = representive_point(cluster_other, transformmatrix, railhead_width)

        if representive_a is None or representive_b is None:
            # Happens in rare cases at switches
            continue

        # Check if both are classified as the same side
        # but a zero and non zero value is not a problem
        if side_a * side_b < 0:
            # Update the cluster with lower confidence of the side estimation
            if np.abs(side_a) < np.abs(side_b):
                representive_a, side_a  = representive_point(cluster, transformmatrix, railhead_width, side=side_b)
            else:
                representive_b, side_b = representive_point(cluster_other, transformmatrix,  railhead_width, side=side_a)

        if representive_a is None or representive_b is None:
            continue

        central = get_central_point(representive_a, representive_b)
        points.append(central)
        points.append(representive_a)
        points.append(representive_b)

    return np.array(points)

In [ ]:
pipeline = pdal.Pipeline([pdal.Reader(file_run24)])
pipeline.execute()
points_run24 = pipeline.arrays[0]
xyz_run24 = np.vstack((points_run24['X'], points_run24['Y'], points_run24['Z'])).T
seed24 = get_seedpoints(points_run24)

pipeline = pdal.Pipeline([pdal.Reader(file_run14)])
pipeline.execute()
points_run14 = pipeline.arrays[0]
seed14 = get_seedpoints(points_run14)
xyz_run14 = np.vstack((points_run14['X'], points_run14['Y'], points_run14['Z'])).T

offset = xyz_run24.mean(axis=0)
xyz_run24 -= offset
xyz_run14 -= offset
seed24 -= offset
seed14 -= offset

cpoints24 = get_points(xyz_run24, seed24)
cpoints14 = get_points(xyz_run14, seed14)


In [5]:
pv24 = pv.PolyData(xyz_run24)
pv14 = pv.PolyData(xyz_run14)
pv24['Intensity'] = points_run24['Intensity']
pv14['Intensity'] = points_run14['Intensity']

p = pv.Plotter()
p.add_mesh(pv24, color="orange",name="Run 24", opacity=0.5, point_size=1)
p.add_mesh(pv14, color="green", name="Run 14", opacity=0.5, point_size=1)

p.show()

Widget(value='<iframe src="http://localhost:41573/index.html?ui=P_0x7fa00425ff40_1&reconnect=auto" class="pyvi…